In [39]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [40]:
#Load SMILES data
data = np.load("ohepsmiles.npz")
data = data["arr_0"]
data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
print("Dataset size: " + str(data.size()))

Dataset size: torch.Size([34131372, 1, 55])


In [41]:
#Get input tensor
def inp(i, shuffle):
    
    #Input (does not include last character in SMILES)
    inp = data[int(shuffle[i] * seq_length) : int((shuffle[i] * seq_length) + seq_length - 2), :, :]

    return inp

In [42]:
#Get target tensor
def target(i, shuffle):
    
    #Target (does not include first character in SMILES)
    target = data[int((shuffle[i] * seq_length) + 1) : int((shuffle[i] * seq_length) + seq_length - 1), :, :]
        
    return target

In [64]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
    
    #Define forward propagation
    def forward(self, inp, hidden, cell):
        output, hidden = self.lstm((inp, (hidden, cell)))
        output = self.linear(output)
        
        return output, hidden
        
    #Define initial hidden states, initialize cell states
    def init_states(self, num_layers, hidden_size):
        hidden = Variable(torch.zeros(num_layers, 1, hidden_size))
        cell = Variable(torch.zeros(num_layers, 1, hidden_size))
        
        return hidden, cell
        
        

In [70]:
#Define training
def train(epochs):
    
    #Initialize hidden and cell states
    hidden, cell = model.init_states(num_layers, hidden_size)
    hidden = hidden.cuda()
    cell = cell.cuda()
    
    #Iterate over desired number of epochs 
    for e in range(epochs):
        #Get random order of SMILES molecules (shuffle data)
        shuffle = np.arange(np.shape(data)[0] / seq_length)
        random.shuffle(shuffle)
        
        #Set initial gradients
        model.zero_grad()
    
        #Set initial loss
        loss = 0 
        
        #Iterate over each molecule in dataset
        for i in range(int(np.shape(data)[0] / seq_length)):
            
            #Iterate over each characeter
            for c in range(seq_length):
                
                #Run model, calculate loss
                output, hidden = model(inp(i, shuffle)[c, :, :], hidden, cell)
                loss += criterion(output, target[c, :, :])
                
                #Backpropagate loss
                loss.backward()
                optimizer.step()
                
            print(loss.data[0] / seq_length)
                

In [71]:
#Initialize model parameters
input_size = np.shape(data)[1]
hidden_size = 256
num_layers = 3
dropout = 0.2
learning_rate = 0.001
epochs = 10
seq_length = 76

In [72]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [73]:
#Run on GPU
model = model.cuda()
criterion = criterion.cuda()

In [74]:
#Train
train(epochs)

AttributeError: 'tuple' object has no attribute 'size'